In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('movies_updated.csv')

In [3]:
df=data[['name','genre','director','writer','star']]

In [4]:
grouped_df = df.groupby('name').agg(lambda x: x.tolist()).reset_index()

In [5]:
grouped_df['Movie ID'] = range(1, len(grouped_df) + 1)

In [6]:
columns_to_join = ['genre', 'director', 'writer','star']  # Add more columns if needed

# for column in columns_to_join:
#     grouped_df[column] = grouped_df[column].apply(lambda x: ' '.join(set(x)))
for column in columns_to_join:
    grouped_df[column] = grouped_df[column].apply(lambda x: ' '.join(set(map(str, x))) if isinstance(x, list) else str(x))

In [7]:
grouped_df['tags']=grouped_df['name']+grouped_df['genre']+grouped_df['director']+grouped_df['writer']+grouped_df['star']

In [8]:
new_df=grouped_df[['Movie ID','name','tags']]

In [9]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\Anjali Aa\AppData\Local\Temp\ipykernel_10484\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [12]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [13]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [14]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [15]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Anjali Aa\AppData\Local\Temp\ipykernel_10484\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [18]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
cosine_similarity(vectors)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [21]:
similarity=cosine_similarity(vectors)

In [22]:
def recommend(movie):
    movie_index=new_df[new_df['name']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]][1])
#         print(new_df.iloc[i[0]].name)

In [23]:
recommend('Hamlet')

Braveheart
Conspiracy Theory
Henry V
Much Ado About Nothing
Pocahontas


In [24]:
# new_df.iloc[576]

In [25]:
# new_df.iloc[785]

In [26]:
# df[df['name']=='Dead Again']

In [27]:
recommend('Innocent Blood')

Nikita
An Innocent Man
'Round Midnight
'night, Mother
*batteries not included


In [28]:
new_df.to_dict()

{'Movie ID': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24,
  24: 25,
  25: 26,
  26: 27,
  27: 28,
  28: 29,
  29: 30,
  30: 31,
  31: 32,
  32: 33,
  33: 34,
  34: 35,
  35: 36,
  36: 37,
  37: 38,
  38: 39,
  39: 40,
  40: 41,
  41: 42,
  42: 43,
  43: 44,
  44: 45,
  45: 46,
  46: 47,
  47: 48,
  48: 49,
  49: 50,
  50: 51,
  51: 52,
  52: 53,
  53: 54,
  54: 55,
  55: 56,
  56: 57,
  57: 58,
  58: 59,
  59: 60,
  60: 61,
  61: 62,
  62: 63,
  63: 64,
  64: 65,
  65: 66,
  66: 67,
  67: 68,
  68: 69,
  69: 70,
  70: 71,
  71: 72,
  72: 73,
  73: 74,
  74: 75,
  75: 76,
  76: 77,
  77: 78,
  78: 79,
  79: 80,
  80: 81,
  81: 82,
  82: 83,
  83: 84,
  84: 85,
  85: 86,
  86: 87,
  87: 88,
  88: 89,
  89: 90,
  90: 91,
  91: 92,
  92: 93,
  93: 94,
  94: 95,
  95: 96,
  96: 97,
  97: 98,
  98: 99,
  99: 100,
  100:

In [29]:
import pickle

In [30]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [31]:
pickle.dump(similarity,open('similarity.pkl','wb'))